In [ ]:
# 安裝必要的套件
!pip install transformers opencc-python-reimplemented sentencepiece pymupdf pytesseract
!apt-get install tesseract-ocr
!apt-get install libtesseract-dev

In [ ]:
!huggingface-cli login
# Log in with your huggingface token (Have at least read access to the model)

In [ ]:
# 匯入所需的函式庫
import fitz  # PyMuPDF 用來處理 PDF
import pytesseract  # OCR 處理
from PIL import Image  # 圖片處理
import io
import re  # 正則表達式處理文本
import json
from transformers import AutoTokenizer  # 用來處理 LLaMA Tokenization
from opencc import OpenCC  # 簡繁體轉換

# 初始化 OpenCC 轉換器（簡體轉繁體中文）
cc = OpenCC('s2t')  # 's2t' 表示簡體轉繁體

# 初始化 LLaMA 的 tokenizer
tokenizer = AutoTokenizer.from_pretrained('taide/TAIDE-LX-7B')


# 步驟 1: 從 PDF 中提取文本
def extract_text_from_pdf(pdf_path):
    text = ""
    pdf_document = fitz.open(pdf_path)  # 打開 PDF 檔案

    for page_num in range(pdf_document.page_count):
        page = pdf_document.load_page(page_num)
        text += page.get_text("text")  # 提取每頁的文本
        image_list = page.get_images(full=True)  # 獲取該頁的所有圖片

        for img_index, img in enumerate(image_list):
            xref = img[0]
            base_image = pdf_document.extract_image(xref)
            image_bytes = base_image["image"]
            image_stream = io.BytesIO(image_bytes)
            image = Image.open(image_stream)

            # 使用 pytesseract 進行 OCR （支援簡體中文和英文）
            ocr_result = pytesseract.image_to_string(image, lang='chi_sim+eng')
            text += ocr_result  # 將 OCR 結果加入文本
            image_stream.close()

    pdf_document.close()
    return text


# 步驟 2: 處理數學公式（使用 OCR 提取公式）
def process_math_formulas(image):
    """處理圖片中的數學公式，並使用 OCR 進行識別"""
    ocr_result = pytesseract.image_to_string(image, config='--psm 6')  # psm 6 用於稀疏文本
    return ocr_result

# 步驟 3: 文本清理與正則化
def clean_text(text):
    # 移除多餘的空白與換行符號
    text = re.sub(r'\s+', ' ', text)

    # 去除特殊符號，但保留中文、英文及常用標點符號
    text = re.sub(r'[^\w\s\u4e00-\u9fff\u3000-\u303f\uff00-\uffef]', '', text)

    return text.strip()

# 步驟 4: 全形轉半形
def fullwidth_to_halfwidth(text):
    """將全形字符轉換為半形字符"""
    new_text = ""
    for char in text:
        code = ord(char)
        if 65281 <= code <= 65374:
            new_text += chr(code - 65248)
        else:
            new_text += char
    return new_text

# 步驟 5: 將簡體中文轉為繁體中文
def convert_to_traditional(text):
    return cc.convert(text)


# 步驟 7: Tokenization 使用 TAIDE-LX-7B
def tokenize_text(text):
    tokens = tokenizer.tokenize(text)
    token_ids = tokenizer.convert_tokens_to_ids(tokens)
    return tokens, token_ids

# 步驟 8: 主處理流程
def process_pdf(pdf_path, output_json_path):
    extracted_text = extract_text_from_pdf(pdf_path)

    cleaned_text = clean_text(extracted_text)

    cleaned_text = fullwidth_to_halfwidth(cleaned_text)

    traditional_text = convert_to_traditional(cleaned_text)


    tokens, token_ids = tokenize_text(traditional_text)

    with open(output_json_path, 'w', encoding='utf-8') as f:
        json.dump({
            "original_text": extracted_text,
            "traditional_text": traditional_text,
            "tokens": tokens,
            "token_length": len(tokens),
            "token_ids": token_ids
        }, f, ensure_ascii=False, indent=4)


In [ ]:
# 使用範例:
pdf_path = '/content/sample.pdf'  # 請將此路徑替換為實際的 PDF 檔案路徑
output_json_path = '/content/output_data.json'

# 執行處理流程
process_pdf(pdf_path, output_json_path)